In [10]:
# Este script extrai dados da API RestCountries e salva diretamente em JSON Lines na camada Bronze,
# sem acumular em lista Python. Cada resposta da API é transformada em JSON string e enviada ao Spark.
# Isso evita uso excessivo de memória e garante compatibilidade com Fabric.

import requests
import json
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Camada Bronze - Países") \
    .getOrCreate()

# Lista de códigos de países
code = [
    "AD", "AL", "AT", "BA", "BE", "BG", "BY", "CH", "CY", "CZ",
    "DE", "DK", "EE", "ES", "FI", "FR", "GB", "GR", "HR", "HU",
    "IE", "IS", "IT", "LI", "LT", "LU", "LV", "MC", "MD", "ME",
    "MK", "MT", "NL", "NO", "PL", "PT", "RO", "RS", "RU", "SE",
    "SI", "SK", "SM", "UA", "VA"
]

json_strings = []
for nome in code:
    url = f"https://restcountries.com/v3.1/alpha/{nome}"
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        # A API pode retornar lista ou objeto único
        if isinstance(data, list):
            for item in data:
                json_strings.append(json.dumps(item))
        else:
            json_strings.append(json.dumps(data))
    else:
        print(f"Erro ao buscar {nome}: {r.status_code}")


json_rdd = spark.sparkContext.parallelize(json_strings)
df = spark.read.json(json_rdd)

df.printSchema()
df.show(5, truncate=False)


df.write.mode("overwrite").json("Files/paises_json")

df_bronze = spark.read.json("Files/paises_json")
df_bronze.printSchema()
df_bronze.show(5, truncate=False)


df.write.mode("overwrite").parquet("Files/paises_parquet")


StatementMeta(, 2e484a95-4eb4-4321-8f8b-65359b813ac6, 12, Finished, Available, Finished)

root
 |-- altSpellings: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area: double (nullable = true)
 |-- borders: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- capital: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- capitalInfo: struct (nullable = true)
 |    |-- latlng: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- car: struct (nullable = true)
 |    |-- side: string (nullable = true)
 |    |-- signs: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- ccn3: string (nullable = true)
 |-- cioc: string (nullable = true)
 |-- coatOfArms: struct (nullable = true)
 |    |-- png: string (nullable = true)
 |    |-- svg: string (nullable = true)
 |-- continents: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- currencies: struct (nullable